# NeqSim Development Notebook — Live from Classpath

This notebook loads NeqSim **directly from the project's compiled classes** (`target/classes/`),
so any Java change you compile is instantly available here — no JAR rebuild needed.

### Quick start (copy into any new notebook)
```python
# Cell 1 — start JVM
from neqsim_dev_setup import neqsim_init, neqsim_classes
ns = neqsim_init(recompile=True)

# Cell 2 — import classes (pick one approach)
ns = neqsim_classes(ns)                        # standard set
# OR import only what you need:
ns.MyClass = ns.JClass("neqsim.some.MyClass")  # via ns
MyClass = jpype.JClass("neqsim.some.MyClass")   # or directly
```

### How it works
- **Cell 1** handles JVM lifecycle: compile → start (or restart if already running)
- **Cell 2** imports Java classes — use `neqsim_classes(ns)` for the standard set,
  or `ns.JClass()` / `jpype.JClass()` to cherry-pick individual classes
- Re-run Cell 1 after Java edits → compiles → restarts kernel automatically

In [1]:
from neqsim_dev_setup import neqsim_init, neqsim_classes
ns = neqsim_init(recompile=True)

Compiling... OK
Classpath:
  1. C:\Users\ESOL\Documents\GitHub\neqsim2\target\classes
  2. C:\Users\ESOL\Documents\GitHub\neqsim2\src\main\resources
  3. C:\Users\ESOL\Documents\GitHub\neqsim2\target\neqsim-3.4.0-shaded.jar

JVM started: C:\Users\ESOL\graalvm\graalvm-jdk-25.0.1+8.1\bin\server\jvm.dll
Ready — call neqsim_classes(ns) to import classes


In [2]:
# Import standard NeqSim classes — add or remove as needed per notebook
ns = neqsim_classes(ns)

All NeqSim classes imported OK


---
### Alternative: import classes directly in the notebook

Instead of `neqsim_classes(ns)` you can import exactly the classes you need
using `ns.JClass()` (or `jpype.JClass()`) — handy when you only need a few,
or need classes not in the standard set.

Since the full classpath is loaded, **any** Java class is accessible,
including inner classes, enums, and standard library types:

In [3]:
# Option A: add to the ns namespace (keeps everything in one place)
ns.DistillationColumn = ns.JClass("neqsim.process.equipment.distillation.DistillationColumn")
ns.Expander = ns.JClass("neqsim.process.equipment.expander.Expander")

# Option B: import into local variables (no ns. prefix needed)
import jpype
SystemUMRPRUMCEos = jpype.JClass("neqsim.thermo.system.SystemUMRPRUMCEos")

# Java standard library classes work too
ArrayList = ns.JClass("java.util.ArrayList")
HashMap = ns.JClass("java.util.HashMap")

# Inner classes / enums use $ notation:
# CapacityRule = ns.JClass("neqsim.some.Outer$InnerClass")

jlist = ArrayList()
jlist.add("methane")
jlist.add("ethane")
print(f"DistillationColumn: {ns.DistillationColumn}")
print(f"SystemUMRPRUMCEos:  {SystemUMRPRUMCEos}")
print(f"Java ArrayList:     {jlist}  size={jlist.size()}")

DistillationColumn: <java class 'neqsim.process.equipment.distillation.DistillationColumn'>
SystemUMRPRUMCEos:  <java class 'neqsim.thermo.system.SystemUMRPRUMCEos'>
Java ArrayList:     [methane, ethane]  size=2


---
## Quick Smoke Test

Create a simple fluid and flash it to confirm everything works.

In [4]:
# ── Create a test fluid (SRK EOS, 25 °C, 60 bara) ──
fluid = ns.SystemSrkEos(273.15 + 25.0, 60.0)
fluid.addComponent("methane", 0.80)
fluid.addComponent("ethane", 0.10)
fluid.addComponent("propane", 0.05)
fluid.addComponent("n-butane", 0.03)
fluid.addComponent("n-pentane", 0.02)
fluid.setMixingRule("classic")
fluid.setMultiPhaseCheck(True)

ops = ns.ThermodynamicOperations(fluid)
ops.TPflash()
fluid.initProperties()

print(f"Temperature : {fluid.getTemperature() - 273.15:.1f} °C")
print(f"Pressure    : {fluid.getPressure():.1f} bara")
print(f"Phases      : {fluid.getNumberOfPhases()}")
print(f"Gas density : {fluid.getPhase('gas').getDensity('kg/m3'):.2f} kg/m³")
print(f"Z-factor    : {fluid.getPhase('gas').getZ():.4f}")

Temperature : 25.0 °C
Pressure    : 60.0 bara
Phases      : 2
Gas density : 62.10 kg/m³
Z-factor    : 0.8145


---
## Process Simulation Example

A minimal process: feed → separator → compressor.

In [5]:
# ── Build a small process ──
process = ns.ProcessSystem()

feed = ns.Stream("feed", fluid)
feed.setFlowRate(50000.0, "kg/hr")
feed.setTemperature(30.0, "C")
feed.setPressure(60.0, "bara")
process.add(feed)

sep = ns.Separator("HP Sep", feed)
process.add(sep)

gas_out = sep.getGasOutStream()

comp = ns.Compressor("Export Comp", gas_out)
comp.setOutletPressure(120.0)
process.add(comp)

process.run()

print(f"Compressor power   : {comp.getPower('kW'):,.1f} kW")
print(f"Outlet temperature : {comp.getOutletStream().getTemperature() - 273.15:.1f} °C")
print(f"Outlet pressure    : {comp.getOutletStream().getPressure():.1f} bara")

Compressor power   : 1,024.0 kW
Outlet temperature : 79.7 °C
Outlet pressure    : 120.0 bara
